In [2]:
# %pip install pandas
# %pip install numpy
# %pip install contractions
# %pip install bs4
# %pip install nltk
# %pip install unidecode
# %pip install matplotlib

  Using cached matplotlib-3.8.3-cp310-cp310-macosx_11_0_arm64.whl.metadata (5.8 kB)
  Using cached contourpy-1.2.0-cp310-cp310-macosx_11_0_arm64.whl.metadata (5.8 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.4/159.4 kB 3.1 MB/s eta 0:00:00a 0:00:01
  Using cached kiwisolver-1.4.5-cp310-cp310-macosx_11_0_arm64.whl.metadata (6.4 kB)
  Using cached pillow-10.2.0-cp310-cp310-macosx_11_0_arm64.whl.metadata (9.7 kB)
  Using cached pyparsing-3.1.2-py3-none-any.whl.metadata (5.1 kB)
Using cached matplotlib-3.8.3-cp310-cp310-macosx_11_0_arm64.whl (7.5 MB)
Using cached contourpy-1.2.0-cp310-cp310-macosx_11_0_arm64.whl (242 kB)
Using cached cycler-0.12.1-py3-none-any.whl (8.3 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 4.2 MB/s eta 0:00:0000:0100:01
Using cached kiwisolver-1.4.5-cp310-cp310-macosx_11_0_arm64.whl (66 kB)
Using cached pillow-10.2.0-cp310-cp310-macosx_11_0_arm64.whl (3.3 MB)
Using cached pyparsing

# Importing Libraries

In [3]:
import pandas as pd
import gzip
import json
import re
from bs4 import BeautifulSoup
from unidecode import unidecode
import nltk
import contractions
import matplotlib.pyplot as plt
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
# from acronym import Acronym


# Df loading function

In [4]:
def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield json.loads(l)

def getDF(path):
  i = 0
  df = {}
  folder = parse(path)
  for d in folder:
    print(i)
    df[i] = d
    i += 1
  return df



In [ ]:
df = getDF('Electronics_5.json.gz')
data_frame = pd.DataFrame.from_dict(df, orient='index')
data_frame.to_csv('data_frame.csv', index=False)

In [ ]:
df = getDF('meta_Electronics.json.gz')
meta_df = pd.DataFrame.from_dict(df, orient='index')
meta_df.to_csv('meta_data.csv', index=False, escapechar='\\')

In [3]:
data = pd.read_csv("data_frame.csv")
meta_data = pd.read_csv("meta_data.csv")

/var/folders/4f/kjgrgc293tn7cn556xk1h1g40000gn/T/ipykernel_15093/1755724970.py:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("data_frame.csv")
/var/folders/4f/kjgrgc293tn7cn556xk1h1g40000gn/T/ipykernel_15093/1755724970.py:2: DtypeWarning: Columns (3,6,14) have mixed types. Specify dtype option on import or set low_memory=False.
  meta_data = pd.read_csv("meta_data.csv")


In [25]:
product_indices = []
product = 'USB Cables'
for i in range(len(meta_data)):
    if product in meta_data['category'][i]:
        product_indices.append(i)

product_meta_data_df = meta_data.iloc[product_indices]
product_meta_data_df = product_meta_data_df.dropna(subset=['title', 'asin'])
product_meta_data_df = product_meta_data_df.drop_duplicates()

product_df= pd.merge(data, product_meta_data_df, on='asin', how='inner')
product_df = product_df.dropna(subset=['overall'])
product_df = product_df.drop_duplicates()

14310

In [ ]:

def expand_acronyms(text):
    expanded_words = []    
    for word in text.split():
        expanded_words.append(contractions.fix(word))  
    expanded_text = ' '.join(expanded_words)
    return expanded_text
def remove_html_tags(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()
def remove_accented_chars(text):
    return unidecode(text)
def remove_special_characters(text):
    return re.sub(r'[^a-zA-Z\s]', '', text)
def lemmatize_text(text):
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    preprocessed_text = ' '.join(tokens)
    return preprocessed_text
def normalize_text(text):
    text = text.lower()
    text = expand_acronyms(text)
    text = remove_html_tags(text)
    text = remove_accented_chars(text)
    text = remove_special_characters(text)
    text = lemmatize_text(text)
    return text

product_df['reviewText'] = product_df['reviewText'].astype(str)
for i in range(len(product_df)):
    print(i)
    pre_text =  normalize_text(product_df['reviewText'].iloc[i])
    product_df.loc[i, "reviewText"]=pre_text
    
product_df.to_csv('product.csv', index=False)


In [10]:
product_df = pd.read_csv("product.csv")
product_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 124237 entries, 0 to 124236
Data columns (total 30 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   overall          120806 non-null  float64
 1   vote             7329 non-null    object 
 2   verified         120806 non-null  object 
 3   reviewTime       120806 non-null  object 
 4   reviewerID       120806 non-null  object 
 5   asin             120806 non-null  object 
 6   style            68955 non-null   object 
 7   reviewerName     120775 non-null  object 
 8   reviewText       124146 non-null  object 
 9   summary          120784 non-null  object 
 10  unixReviewTime   120806 non-null  float64
 11  image            1181 non-null    object 
 12  category         120806 non-null  object 
 13  tech1            51449 non-null   object 
 14  description      120806 non-null  object 
 15  fit              0 non-null       float64
 16  title            120806 non-null  obje

/var/folders/4f/kjgrgc293tn7cn556xk1h1g40000gn/T/ipykernel_16371/896985281.py:1: DtypeWarning: Columns (1,2) have mixed types. Specify dtype option on import or set low_memory=False.
  product_df = pd.read_csv("product.csv")


In [8]:
total_reviews = product_df['reviewText'].count()
average_rating = product_df['overall'].mean()
num_unique_products = product_df['asin'].nunique()
good_ratings = product_df[product_df['overall'] >= 3]['overall'].count()
bad_ratings = product_df[product_df['overall'] < 3]['overall'].count()
rating_counts = product_df['overall'].value_counts().sort_index()

# Display the results
print(" . Number of rows for the product:", len(product_df))
print("a. Number of Reviews:", total_reviews)
print("b. Average Rating Score:", average_rating)
print("c. Number of Unique Products:", num_unique_products)
print("d. Number of Good Ratings:", good_ratings)
print("e. Number of Bad Ratings:", bad_ratings)
print("f. Number of Reviews corresponding to each Rating:")
print(rating_counts)


 . Number of rows for the product: 124237
a. Number of Reviews: 124146
b. Average Rating Score: 4.365503369037961
c. Number of Unique Products: 2591
d. Number of Good Ratings: 107710
e. Number of Bad Ratings: 13096
f. Number of Reviews corresponding to each Rating:
overall
1.0     8621
2.0     4475
3.0     6809
4.0    15124
5.0    85777
Name: count, dtype: int64


In [11]:
import warnings
import tqdm

# Raise all warnings as exceptions
warnings.filterwarnings("ignore")

In [ ]:
def generate_wordcloud(text_list, title):
    text = ' '.join(text_list)
    wordcloud = WordCloud(width = 800, height = 800, 
                background_color ='white', 
                stopwords = None, 
                min_font_size = 10).generate(text)

    # Plot the WordCloud image                        
    plt.figure(figsize = (8, 8), facecolor = None) 
    plt.imshow(wordcloud) 
    plt.axis("off") 
    plt.tight_layout(pad = 0) 
    plt.title(title)
  
    plt.show()

In [28]:
top_20_most_reviewed_brands = product_df['brand'].value_counts().head(20)
top_20_least_reviewed_brands = product_df['brand'].value_counts().tail(20)
most_positively_reviewed_headphone = product_df.groupby('title')['overall'].mean().nlargest(1)


print("a. Top 20 most reviewed brands:")
print(top_20_most_reviewed_brands)
print("b. Top 20 least reviewed brands:")
print(top_20_least_reviewed_brands)
print("c. Most positively reviewed headphone:")
print(most_positively_reviewed_headphone)

Most positively reviewed headphone:
title
Myron &amp; Davis AE52 Dual Channel IR headphone    5.0
Name: overall, dtype: float64
a. Top 20 most reviewed brands:
brand
Sony              31260
Sennheiser        21221
Bose               9489
Plantronics        8239
Skullcandy         7862
JVC                7024
JLAB               6977
Audio-Technica     6651
Philips            6458
Panasonic          5980
Koss               5764
Mpow               5643
LG                 5624
Samsung            5597
Bluedio            5364
MEE audio          4644
Anker              4291
Symphonized        4285
TaoTronics         4056
Beats              3993
Name: count, dtype: int64
Top 20 least reviewed brands:
brand
EMPIRE AUDIO USA         5
Cosa Nova                5
MAXELL(R)                5
OAproda                  5
TomTom                   5
tech21                   5
SoulBuddy                5
fizwan                   5
OCR                      5
Spark                    5
VIILER                

In [ ]:
set(product_df['date'])

In [35]:
product_df['date'] = pd.to_datetime(product_df['date'], errors='coerce')
# Count of ratings for the product over 5 consecutive years
ratings_over_years = product_df.set_index('date').groupby(pd.Grouper(freq='Y')).size().head(5)
print("Count of ratings for the product over 5 consecutive years:")
print(ratings_over_years)

/var/folders/4f/kjgrgc293tn7cn556xk1h1g40000gn/T/ipykernel_3520/1072341768.py:3: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  ratings_over_years = product_df.set_index('date').groupby(pd.Grouper(freq='Y')).size().head(5)


Count of ratings for the product over 5 consecutive years:
date
1973-12-31    129
1974-12-31      0
1975-12-31      0
1976-12-31      0
1977-12-31      0
dtype: int64
